# Overview of Terraform Ops Notebook

## How to Use this Notebook
<p>
This notebook shows you how to easily explore Terraform features and functions by directly using the OCI Comand Line Interface (CLI).  First you will need to have the CLI tool installed on your machine and working.  Then you add a few elements to your config file as noted below, then Authenticate to your tenancy using the notebook cell provided.
</p>


## References


Resource Manager Stacks and Jobs [link](https://docs.oracle.com/en-us/iaas/Content/ResourceManager/Tasks/managingstacksandjobs.htm#stackstate  "Stacks and Jobs")


# Initialize Connection to Tenancy



In [ ]:
import oci
import pprint
from pyUtils.token_signer import token_signer
import json ,  pprint
from IPython.display import display, Math, HTML, Markdown
from pyUtils.runCLI import runCLI
import datetime  
import pytz

display(HTML("<style>.container { width:75% !important; }</style>"))
# display(HTML("<style>div.output_scroll { height: 70em; }</style>"))

try:
    tenancyProfile = oci.config.from_file()['target_profile']
except:
    print('A Target Profile not found in config file.  Using Default Profile')
    tenancyProfile = 'DEFAULT'

tenancyConfig = oci.config.from_file(profile_name=tenancyProfile)

config = ' --profile ' + tenancyConfig['auth_profile'] +  ' --auth security_token'
print('CLI Auth config string is: ', config)
print('Repository URL:', tenancyConfig['source_repository_url'])
print('Repository Git Branch:', tenancyConfig['source_branch_name'])
print('Repository Source Working Directory:', tenancyConfig['source_working_directory'])
print("Auth Profile"   ,tenancyConfig['auth_profile']) 
 
def format_date(fDate) :
    newDate = datetime.datetime.strptime(fDate,"%Y-%m-%dT%H:%M:%S.%f%z")
    return newDate.astimezone(pytz.timezone('US/Eastern')).strftime("%m/%d/%Y %H:%M %p %Z")
   
    

# Terraform CLI


## Stacks

### Stack List 

In [ ]:
command = 'oci resource-manager stack list  --compartment-id ' \
        + tenancyConfig['target_compartment']  + config
print('Command: \n', command)
stdout, stderr = runCLI(command)
print(stderr)
stacks = {}
try:
    stacks = json.loads(stdout)
    print('There were ' + str(len(stacks['data']) )+ '  Stacks Found')
    stackDict = {}
    for stack in stacks['data']:
        print(stack['display-name'])
        print('    Description: '  ,stack['description'])
        print('    Stack ID: ',stack['id'])
        print('    Time Created: ', format_date(stack['time-created'])   )
        print('    Status: ', stack['lifecycle-state'])
        print('    Terraform Version: ', stack['terraform-version'])
        stackDict[stack['display-name']] = stack['id']
        print('--')
except:
    print( 'Json convert error, likely No Data Found' ,stdout)
# pprint.pprint(stack)

### Get Stack (set target stack)

In [ ]:
targetStack = 'ocid1.ormstack.oc1.iad.aaaaaaaav6tvb7esfmng3dfpy24ah576ttdbqeuf75menzgytuc6plt5aosq'
stackId = targetStack
command = ' oci resource-manager stack get --stack-id  ' + stackId +  config
print('Command: \n', command)
stdout, stderr = runCLI(command)
print(stderr)
stack = ''
# try:
stack = json.loads(stdout)
print(stack['data']['display-name'])
try:
    print('    Git Branch: ',stack['data']['config-source']['branch-name'])
    print('    Git Directory: ',stack['data']['config-source']['working-directory'])
except:
    print('Not in Github')
print('    Status: ',stack['data']['lifecycle-state'])
print('    Time Created: ',  format_date(  stack['data']['time-created']))
print('    Stack ID: ',stack['data']['id'])
# except:
#     print ('Error: ',stdout )
print('\n---------------')
# pprint.pprint(stack)

### Get Stack's State

In [ ]:
file = "\"-\""
# stackId = 'ocid1.ormstack.oc1.iad.aaaaaaaa'
stackId = targetStack
command = 'oci resource-manager stack get-stack-tf-state --stack-id  ' \
          + stackId + ' --file ' + file + config
print('Command: \n', command)
stdout, stderr = runCLI(command)

try:
    stackStates = json.loads(stdout)
except:
    print ('Error: ',stderr )
print('\nFetch State Complete, run Stack Report in cell below !')
# pprint.pprint(stackStates)

### Import Stack State from File

In [ ]:
with open('../../TRUCCR/uat_update_state.json') as f:
    data = f.read()
stackStates = json.loads(data)


### Stack Report

In [ ]:
object_storage_client = oci.object_storage.ObjectStorageClient(
                        {'region':oci.config.from_file(profile_name=tenancyProfile)['region']},
                        signer=token_signer(tenancyConfig['auth_profile'])  )
get_namespace_response = object_storage_client.get_namespace()
resourceMap = {}
resourceMap['Environment-Name']='Integration'
resourceMap['Environment-Prefix']='uat_'
resourceMap['Environment-Descr']='Build Environment to Debug Terraform template'
resourceMap['namespace'] = get_namespace_response.data
envResourceMap = resourceMap
envResourceMap['HC Monitors'] = []
envResourceMap['API Gateways'] = []
envResourceMap['API Gateway Deployments'] = []
envResourceMap['Function Applications'] = []
envResourceMap['Functions'] = []
envResourceMap['Object Storage Buckets'] = []
envResourceMap['Event Rules'] = []
envResourceMap['Logging'] = []
envResourceMap['Vault'] = []

stackOrder = ['httpmonitor', 'apigateway','apideployment', 'fnapp'  , 'fnfunc' 
              ,  resourceMap['namespace'],   'eventrule' ,'vault'  , '.log.'   ]
print('Stack Resource Listing\n-------------\n')
for instType in stackOrder:
    for stackState in stackStates['resources'] :  
#     try:
        for stackStateInstance in stackState['instances']:
            attributeId = stackStateInstance['attributes']['id']
#             print('ATTR *** ', attributeId  )
            if instType in attributeId and  'httpmonitor' ==  instType  :
                print('HTTP Monitor')
                print('    ' ,  stackStateInstance['attributes']['display_name'])
                print('        ID:    ' ,  stackStateInstance['attributes']['id'])
                print('        Target:' ,  stackStateInstance['attributes']['targets'][0])
                print('        Path:  ' ,  stackStateInstance['attributes']['path'])
                myMon = stackStateInstance
                resourceMap[stackStateInstance['attributes']['display_name']] = \
                            stackStateInstance['attributes']['id']
                newDict={}
                newDict[stackStateInstance['attributes']['display_name']] = \
                    {'ID':     stackStateInstance['attributes']['id'],
                     'Targets':stackStateInstance['attributes']['targets'][0],
                     'Path':   stackStateInstance['attributes']['path']  
                    }
                envResourceMap['HC Monitors'].append(newDict)
                continue
            elif instType in attributeId  and  instType == resourceMap['namespace']:
                print('Object Storage Bucket')
                print('    ' , stackStateInstance['attributes']['name'])
                print('        ID:' , stackStateInstance['attributes']['bucket_id'])
                myOs = stackStateInstance
                resourceMap[stackStateInstance['attributes']['name']] = \
                            stackStateInstance['attributes']['bucket_id']
                newDict={}
                newDict[stackStateInstance['attributes']['name']] = \
                    {'ID':     stackStateInstance['attributes']['bucket_id']  }
                envResourceMap['Object Storage Buckets'].append(newDict)
                continue                
            elif instType in attributeId  and  instType ==  'apideployment' :
                print('        API Gateway Deployment')
                print('            ' , stackStateInstance['attributes']['display_name'])
                print('                ID:      ' , stackStateInstance['attributes']['id'])       
                print('                Endpoint:' , stackStateInstance['attributes']['endpoint'])    
                print('                Function:' , stackStateInstance['attributes']['specification'][0]['routes'][0] ['backend'][0]['function_id'])
                deployPath = stackStateInstance['attributes']['path_prefix']  + \
                    stackStateInstance['attributes']['specification'][0]['routes'][0] ['path']   
                print('                Path:    ' , deployPath)
                myDeply = stackStateInstance
                resourceMap[stackStateInstance['attributes']['display_name']] = \
                            stackStateInstance['attributes']['id']
                newDict={}
                newDict[stackStateInstance['attributes']['display_name']] = \
                    {'ID':     stackStateInstance['attributes']['id'],
                     'Endpoint':stackStateInstance['attributes']['endpoint'],
                     'Function':stackStateInstance['attributes']['specification'][0]['routes'][0] ['backend'][0]['function_id'],
                     'Path':   deployPath  
                    }
                envResourceMap['API Gateway Deployments'].append(newDict)
                continue                  
            elif instType in attributeId  and  instType ==  'apigateway' :
                print('API Gateway')
                myGtw = stackStateInstance
                print('    ' , stackStateInstance['attributes']['display_name'])     
                print('        ID:        ' , stackStateInstance['attributes']['id'])                  
                print('        Hostname : ' , stackStateInstance['attributes']['hostname'])          
                print('        IP Address:' , stackStateInstance['attributes']['ip_addresses'][0]['ip_address'])                    
                resourceMap[stackStateInstance['attributes']['display_name']] = \
                            stackStateInstance['attributes']['id']
                newDict={}
                newDict[stackStateInstance['attributes']['display_name']] = \
                    {'ID':     stackStateInstance['attributes']['id'],  
                     'hostname':     stackStateInstance['attributes']['hostname'],  
                     'IP Address':     stackStateInstance['attributes']['ip_addresses'][0]['ip_address']                 
                    }
                envResourceMap['API Gateways'].append(newDict)
                continue                  
            elif instType in attributeId  and  instType ==  '.log.' :
                print('Log')
                print('    ' , stackStateInstance['attributes']['display_name'])
                print('         ID:' , stackStateInstance['attributes']['id'])
                myLog = stackStateInstance
                resourceMap[stackStateInstance['attributes']['display_name']] = \
                            stackStateInstance['attributes']['id']
                resourceMap['truccr-Log-Group'] = \
                            stackStateInstance['attributes']['log_group_id']
                newDict={}
                newDict[stackStateInstance['attributes']['display_name']] = \
                    {'ID':     stackStateInstance['attributes']['id']  }
                envResourceMap['Logging'].append(newDict)
                continue                  
            elif instType in attributeId  and  instType ==  'eventrule' :
                print('Event Rule')
                print('    ' , stackStateInstance['attributes']['display_name'])   
                print('        ID:          ' , stackStateInstance['attributes']['id'])  
                myRule = stackStateInstance
                conditionsStr = stackStateInstance['attributes']   ['condition']
                conditions = json.loads(conditionsStr)
                print('        Bucket Name: ',  conditions['data']['additionalDetails']['bucketName'][0]  )
                print('        Filter  :    ', conditions ['data']['resourceName'][0]     )
                eventTypes = conditions['eventType'] 
                print('        Event Types')
                for eventType in eventTypes:
                    print('            ', eventType)
                resourceMap[stackStateInstance['attributes']['display_name']] = \
                            stackStateInstance['attributes']['id']
                newDict={}
                newDict[stackStateInstance['attributes']['display_name']] = \
                    {'ID':     stackStateInstance['attributes']['id'],
                     'Bucket Name': conditions['data']['additionalDetails']['bucketName'][0] ,
                     'Filter': conditions ['data']['resourceName'][0] ,
                     'Event Types':   eventTypes  
                    }
                envResourceMap['Event Rules'].append(newDict)
                continue                  
            elif instType in attributeId  and  instType ==  'fnapp' :
                print('Application')
                print('    ' , stackStateInstance['attributes']['display_name'])   
                print('        ID:' , stackStateInstance['attributes']['id'])    
                resourceMap[stackStateInstance['attributes']['display_name']] = \
                            stackStateInstance['attributes']['id']
                newDict={}
                newDict[stackStateInstance['attributes']['display_name']] = \
                    {'ID':     stackStateInstance['attributes']['id']           
                    }
                envResourceMap['Function Applications'].append(newDict)
                continue  
            elif instType in attributeId  and  instType ==  'fnfunc' :
                print('        Function')
                print('            ' , stackStateInstance['attributes']['display_name'])    
                print('              ID:     ' , stackStateInstance['attributes']['id'])
                print( '              Image:  '  ,stackStateInstance['attributes']['image'])
                configDict = stackStateInstance['attributes']['config']
                myFunct = stackStateInstance
                if len(configDict.keys()) > 0:
                    print('               Configuration:')
                    [print( '                    ' + key,':',value) for key, value in configDict.items()]
                resourceMap[stackStateInstance['attributes']['display_name']] = \
                            stackStateInstance['attributes']['id']
                newDict={}
                newDict[stackStateInstance['attributes']['display_name']] = \
                    {'ID':     stackStateInstance['attributes']['id'],  
                     'Image':     stackStateInstance['attributes']['image'],  
                     'Configuration':  configDict,                 
                     'Application ID': stackStateInstance['attributes']['application_id']
                    }
                envResourceMap['Functions'].append(newDict)
            elif instType in attributeId  and  instType ==  'vault' :
                print('        Key Vault')
                print('            ' , stackStateInstance['attributes']['display_name'])    
                print('              ID:     ' , stackStateInstance['attributes']['id'])
                myVaults = stackStateInstance
                resourceMap[stackStateInstance['attributes']['display_name']] = \
                            stackStateInstance['attributes']['id']
                newDict={}
                newDict[stackStateInstance['attributes']['display_name']] = \
                    {'ID':     stackStateInstance['attributes']['id']  }
                envResourceMap['Vault'].append(newDict)
                continue                       
    print()

print('-------------')
with open('TFResourceMap.json', 'w') as convert_file:
     convert_file.write(json.dumps(resourceMap))
print('Conversion of Resource Map Complete - TFResourceMap.json'  )
with open('envResourceMap.json', 'w') as convert_file:
     convert_file.write(json.dumps(envResourceMap))
print('Conversion of Environment Resource Map Complete - envResourceMap.json ' )
#     pprint.pprint (stackState['instances'])
print('DONE')

In [ ]:
pprint.pprint(envResourceMap)


In [ ]:
with open('TFResourceMap.json') as f:
    data = f.read()

# reconstructing the data as a dictionary
rMap = json.loads(data)
for k,v in rMap.items():
    print(k,v)
# print(js)

### Stack from file

<div class="alert alert-block alert-info">
<b>Tip:</b> To Check if Stack creation has completed, re-run the Stack List Report.</div>


In [ ]:
displayName = '\"My Display Name\"'
stackDescription = '\"Whatever \"'
configSource = '\"../terraform_files/Template.zip\"'
command = 'oci resource-manager  stack create --compartment-id  ' + targetComptId \
          + ' --config-source ' + configSource \
          + ' --description ' + stackDescription \
          + ' --display-name ' + displayName \
          + ' --terraform-version  "1.0.x" ' \
          +  config
print('Command: \n', command)
stdout, stderr = runCLI(command)
print(stderr)
try:
    stackOut = json.loads(stdout)
except:
    print ('Error: ',stdout )

print('Done')

### Stack from Template

In [ ]:
# variables =  '"{\\"Env\\": \\"Blue\\", \\"Log_group_id\\": \\"cecece\\"}"'
displayName = '\" Stack 12/23 4pm \"'
sourceWorkingDirectory = ''
stackDescription = '\"This Stack included .DS_Store files\"'
templateId = ''
command = 'oci resource-manager stack create-from-template ' \
        + ' --compartment-id ' + targetComptId \
        + ' --description ' + stackDescription \
        + ' --display-name ' + displayName \
        + ' --template-id ' + templateId  \
        + ' --terraform-version  "1.0.x" ' \
        + config
#         + ' --variables ' + variables  \
#         + ' --working-directory ' + sourceWorkingDirectory \

print('Command: \n', command)
stdout, stderr = runCLI(command)
print(stderr)
try:
    stackOut = json.loads(stdout)
except:
    print ('Error: ',stdout )

print('Done')

### Stack from Github

#### Required Environment Config

In [ ]:


devEnvMap = {}
devEnvMap['Environment'] = 'dev'
devEnvMap['Log_Group_Id'] = 'ocid1.loggroup.oc1.iad.amaaaaaafmyzdhaavre2ja7dbs43o3lhimbiqsjnbiht3s5lviso3flccayq'

CiEnvMap = {}
CiEnvMap['Environment'] = 'CI'
CiEnvMap['Log_Group_Id'] = 'ocid1.loggroup.oc1.iad.lllll'


preProdEnvMap = {}
preProdEnvMap['Environment'] = ''
preProdEnvMap['Log_Group_Id'] = ''
preProdEnvMap['user_id'] = 'ocid1.user.oc1..aaaaaaaa6e3d5pgps3hqvgivzutwzuhsz7ouh7dsxlut4ztvshtmg76rlyzq'
preProdEnvMap['subnet_id'] = 'ocid1.subnet.oc1.iad.aaaaaaaagtennw3dsq3vtevwfjh6tqgncuebcqdhtefcms3ipksisvdae7sa'
preProdEnvMap['subnet_id_pub'] = 'ocid1.subnet.oc1.iad.aaaaaaaaetvjz3pboyo4hpoibyaneoy7ht7sosl5ylzyvd765624p5ptisja'
preProdEnvMap['compartment_ids'] = 'ocid1.compartment.oc1..aaaaaaaarfur65hsjpoxtgcc6caxjj25tg4awl4acavefg2hmbyuf2shutrq'

prodEnvMap = {}
prodEnvMap['Environment'] = ''
prodEnvMap['Log_Group_Id'] = ''
prodEnvMap['user_id'] = 'ocid1.user.oc1..NOT_USED'
prodEnvMap['subnet_id'] = 'ocid1.subnet.oc1.iad.aaaaaaaa7coiwqsynqessfbzf3dxh54voeotrlpr2xnlomy6wdsdaodngn4q'
prodEnvMap['subnet_id_pub'] = 'ocid1.subnet.oc1.iad.aaaaaaaa2bnsclqyfc6imzemmzaq2fyshdnevvorbvewlp3665nmqv3edepq'
prodEnvMap['compartment_ids'] = 'ocid1.compartment.oc1..aaaaaaaaduokjnd7r6g74kaliyjuxxtfbmlkatyqbwc6nljbmdeyimbyss3q'

convEnvMap = {}
convEnvMap['Environment'] = '$env'
convEnvMap['Log_Group_Id'] = '$log_group_id'
convEnvMap['subnet_id'] = '$subnet_id'
convEnvMap['subnet_id_pub'] = '$subnet_id_pub'
convEnvMap['compartment_ids'] = '$compt_id'
convEnvMap['user_id'] = '$user_id'
for k,v in prodEnvMap.items():
    print('export ' +  k.replace('$','') + '='+v)

#### Create Stack

In [ ]:
displayName = '\"dev environment - branch: alpha-dev \"'
stackDescription = '\"This stack was initialy created from the alpha-dev branch. \"'
branchName = tenancyConfig['source_branch_name']
branchName = 'alpha-dev'
workingDir = tenancyConfig['source_working_directory']

envMap = preProdEnvMap
envMap['Environment'] = devEnvMap['Environment']
envMap['Log_Group_Id'] = devEnvMap['Log_Group_Id']

variables = '"{\\"env\\":\\"ENV\\",\\"log_group_id\\":\\"LGID\\" ,\\"user_id\\":\\"USRID\\" ' + \
    ',\\"subnet_id\\":\\"SUBNET\\",\\"subnet_id_pub\\":\\"PSUBNT\\" ,\\"compartment_ids\\":\\"COMPT\\" }"'
variables = variables.replace('ENV',envMap['Environment'] )
variables = variables.replace('LGID',envMap['Log_Group_Id'] )
variables = variables.replace('USRID',envMap['user_id'] )
variables = variables.replace('SUBNET',envMap['subnet_id'] )
variables = variables.replace('PSUBNT',envMap['subnet_id_pub'] )
variables = variables.replace('COMPT',envMap['compartment_ids'] )

command = 'oci resource-manager  stack create-from-git-provider --compartment-id  ' \
          + tenancyConfig['target_compartment'] \
          + ' --config-source-configuration-source-provider-id ' + tenancyConfig['github_source_provider']  \
          + ' --config-source-repository-url ' + tenancyConfig['source_repository_url'] \
          + ' --config-source-branch-name ' + branchName \
          + ' --description ' + stackDescription \
          + ' --display-name ' + displayName \
          + ' --config-source-working-directory ' + workingDir \
          + ' --terraform-version  "1.0.x" ' \
          + ' --variables ' + variables \
          +  config
 
print('Command: \n', command)
# x = sssak
stdout, stderr = runCLI(command)
print(stderr)
try:
    stackOut = json.loads(stdout)
except:
    print ('Error: ',stdout )

targetStack = stackOut['data']['id']
print('Target Stack Name is:',stackOut['data']['display-name'] )
print('                  ID:',targetStack)
print('---------------')
# pprint.pprint(stackOut)
print('Done')

### Update Stack from Github

In [ ]:
# displayName = '\"dev - Service Transmission Test - Junk \"'
stackDescription = '\"Updated Stack to correct bucket names per iac issue #20  \"'
branchName = 'alpha-dev-i20'
targetStack = 'asfd'
command = 'oci resource-manager  stack update-from-git-provider '  \
          + ' --config-source-configuration-source-provider-id ' + tenancyConfig['github_source_provider']  \
          + ' --config-source-repository-url ' + tenancyConfig['source_repository_url'] \
          + ' --config-source-branch-name ' + branchName \
          + ' --stack-id ' + targetStack \
          + ' --description ' + stackDescription \
          + ' --config-source-working-directory ' + tenancyConfig['source_working_directory'] \
          + ' --terraform-version  "1.0.x" ' \
          +  config
 
print('Command: \n', command)
# stdout, stderr = runCLI(command)
# print(stderr)
# try:
#     stackOut = json.loads(stdout)
# except:
#     print ('Error: ',stdout )

# targetStack = stackOut['data']['id']
# print('Target Stack Name is:',stackOut['data']['display-name'] )
# print('     ID:',targetStack)
# print('---------------')
# # pprint.pprint(stackOut)
# print('Done')

### Purge Stacks

In [ ]:

stackPurgeList = []
for name,id  in stackDict.items():
    if  'devop'  in name :
        print(name, ' - ' + id)
        stackPurgeList.append(id)
        print('---')
print('\nNumber of stacks in List: ',len(stackPurgeList))

In [ ]:
print('Beginning Purge')
for stack2Purge in stackPurgeList:
    command = 'oci resource-manager stack delete --force --stack-id ' + stack2Purge + config
    print('Command: \n', command)
    stdout, stderr = runCLI(command)
    print(stderr)
    print('Out: ', stdout)
print('Done')

## Jobs

### Stack Jobs

In [ ]:
# stackId = 'ocid1.ormstack.oc1.iad.aaaaaaaarhxdrh5sjet6cw53m7c3bgt7rivncd2bc4juvzotvovkzlgul7aa'
stackId = targetStack
command = 'oci resource-manager job list  --stack-id ' + stackId  + config
print('Command: \n', command)
stdout, stderr = runCLI(command)
print(stderr)
stackJobs = ''
try:
    stackJobs = json.loads(stdout)
except:
    print( 'Json convert error' ,stdout)
try:
    for job in stackJobs['data']:
        print(job['display-name'])
        print('    Operation: ', job['operation'])
        print('    Status: ' , job['lifecycle-state'])
        print('    ID: ', job['id'])
        print('    Time Created: ',format_date (job['time-created']))
        print('    Time Finished: ',format_date (job['time-finished']))
        print('--')
except:
    print('No Records')
print('----------\n')
# pprint.pprint(stackJobs['data'])
print('Done')

### Stack Plan

In [ ]:
# targetStack = 'ocid1.ormstack.oc1.iad.xxxxx'
stackId = targetStack
displayName = '\"Plan for alpha-dev-i15 update \"'
# variables =  '"{\\"env\\": \\"Blue\\", \\"log_group_id\\": \\"cecece\\"}"'
command = ' oci resource-manager job create-plan-job --stack-id  ' + stackId \
        + ' --display-name ' + displayName  \
        +  config
#         + ' --variables ' + variables \
print('Command: \n', command)
stdout, stderr = runCLI(command)
print(stderr)
print('Creation of Plan Job Completed!!')
try:
    stackPlan = json.loads(stdout)
    print(stackPlan['data']['display-name'])
    print('    Git Branch: ',stackPlan['data']['config-source']['branch-name'])
    print('    Git Directory: ',stackPlan['data']['working-directory'])
    print('    Terraform Operation: ',stackPlan['data']['operation'])
    print('    Status: ',stackPlan['data']['lifecycle-state'])
    print('    Time Created: ',  format_date (stackPlan['data']['time-created']))
    print('    Job ID: ',stackPlan['data']['id'])
    print('    Stack ID: ',stackPlan['data']['stack-id'])
    targetJob = stackPlan['data']['id']
except:
    print ('Error: ',stdout )
print('\n---------------')
print('Target Stack now is: ', targetStack )
print('\nTarget Job ID is: '+ targetJob + '\n\n')
# pprint.pprint(stackPlan)
print('Done')

### Job Log Tail

In [ ]:
# jobId = 'ocid1.ormjob.oc1.iad.aaaaaaaamf7tmrhb6akogyxufutbdibgczsxvc7eip4zzubsrlsokfi5qrjq'
jobId = targetJob
command = 'oci resource-manager job  get-job-logs-content  --job-id ' + jobId + config

print('Command: \n', command)
stdout, stderr = runCLI(command)
print(stderr)
# print('Creation of Plan Job Completed!!')
logCutoff = 4000
try:
    jobLog = json.loads(stdout)
    logContent = jobLog['data']
    logLength = len(logContent)
    print('Log Length is: ' ,logLength)
    if logLength > logCutoff :
        print(logContent[logLength -logCutoff: logLength ])
    else:
        print(logContent)

except:
    print ('Error reading Log: ' )

print('\n---------------')


### Stack Apply

In [ ]:
stackId = targetStack
displayName = '\"Initial Apply for i26 Update \"'
execPlanStrategy = 'AUTO_APPROVED'
# stackId = ''
command = ' oci resource-manager job create-apply-job --stack-id  ' + stackId \
          + ' --display-name ' + displayName  \
          + ' --execution-plan-strategy ' + execPlanStrategy  \
          +  config
print('Command: \n', command)
stdout, stderr = runCLI(command)
print(stderr)
try:
    stackPlan = json.loads(stdout)
    print(stackPlan['data']['display-name'])
    print('    Git Branch: ',stackPlan['data']['config-source']['branch-name'])
    print('    Git Directory: ',stackPlan['data']['working-directory'])
    print('    Terraform Operation: ',stackPlan['data']['operation'])
    print('    Status: ',stackPlan['data']['lifecycle-state'])
    print('    Time Created: ',format_date( stackPlan['data']['time-created']))
    print('    Job ID: ',stackPlan['data']['id'])
    print('    Stack ID: ',stackPlan['data']['stack-id'])
    targetJob = stackPlan['data']['id']
except:
    print ('Error: ',stdout )
print('\nTarget Job ID is: '+ targetJob + '\n\n')
print('Done')



### Stack Destroy

<div class="alert alert-block alert-danger">
<b>Danger:</b> Running Stack Destroy will permanently remove OCI components.  Use caution when using.
</div>

In [ ]:
execPlanStrategy = 'AUTO_APPROVED'
stackId = ''
stackId = targetStack
command = ' oci resource-manager job create-destroy-job --stack-id  ' \
           + stackId  +  ' --execution-plan-strategy '  + execPlanStrategy  +     config
print('Do you want to destroy?(yes/no): ', stackId)
a = input()
if a == 'yes':
    print('yessire')
    print('Command: \n', command)
    stdout, stderr = runCLI(command)
    stackDestroy = json.loads(stdout)
    targetJob = stackDestroy['data']['id']
    print('\nTarget Job ID is: '+ targetJob + '\n\n')
    print(stderr)
#     print(stdout)
else:
    print('Destroy Stack Cancelled !')

### Purge Jobs

**TBD**

## Templates

### Template List

In [ ]:
sortBy = ' --sort-by TIMECREATED --sort-order DESC  '
command = 'oci resource-manager template list --template-category-id 3 -c ' \
          + tenancyConfig['target_compartment'] + sortBy + config
print('Command: \n', command)
stdout, stderr = runCLI(command)
print(stderr)
templates = json.loads(stdout)

print('There were ' + str(len(templates['data']['items']) )+ '  Templates Found\n -------')
templatesDict = {}
for template in templates['data']['items']:
    print(template['display-name'])
    print('    Description: ', template['description'] )
    print('    ID: ', template['id']  + '\n ---')
    templatesDict[template['display-name']] = template['id']
# pprint.pprint(templates['data'])

### Get Template

In [ ]:
templateId = 'ocid1.ormtemplate.oc1.iad.mmmmmm'
command = 'oci resource-manager template get --template-id  ' + templateId  + config
print('Command: \n', command)
stdout, stderr = runCLI(command)
print(stderr)
template = json.loads(stdout)

pprint.pprint(template['data'])

### Get Template Config File

In [ ]:

file = "\"templatex.zip\""
templateId = 'template_id'
command = ' oci resource-manager template get-template-tf-config --template-id '+ templateId  + ' --file ' + file + config
print('Command: \n', command)
stdout,stderr = runCLI(command)
print(stderr)
try:
    template = json.loads(stdout)
except:
    print ('Error: ',stdout )

print('Done')

### Create Private Template

In [ ]:
# config1 = from_file(file_location="")
import base64
with open( r"Template.zip",'rb' ) as file:
    zipContents = file.read()
encodedZip = base64.b64encode(zipContents).decode('ascii')

zipfileupload = oci.resource_manager.models.CreateTemplateZipUploadConfigSourceDetails()
zipfileupload.config_source_type = 'ZIP_UPLOAD'
# zipfileupload.working_directory = 'MyTerraformScripts'
zipfileupload.zip_file_base64_encoded = encodedZip

resourcemanager = oci.resource_manager.models.CreateTemplateDetails()
resourcemanager.compartment_id = tenancyConfig['target_compartment']
resourcemanager.display_name = "Sample Template2"
resourcemanager.description = "Sample Temaplate2"
resourcemanager.template_config_source = zipfileupload

# def getCommandOutput(pCommand):
# Stack = oci.resource_manager.ResourceManagerClient(config1)
Template = oci.resource_manager.ResourceManagerClient({'region': tenancyConfig['region']},
                                          signer=get_signer(tenancyConfig['auth_profile'] ))

Template.create_template(create_template_details=resourcemanager)


### Purge Templates

In [ ]:
templatePurgeList = []
for name,id  in templatesDict.items():
    if  1==1 : #'Sample' in name :
        print(name, ' - ' + id)
        templatePurgeList.append(id)
print('\nNumber of stacks in List: ',len(templatePurgeList))

In [ ]:
print('Beginning Purge of Templates')
for template2Purge in templatePurgeList:
    command = 'oci resource-manager template delete --force --template-id ' + template2Purge + config
    print('Command: \n', command)
    stdout, stderr = runCLI(command)
    print(stderr)
    print('Out: ', stdout)
print('Done')

# Cleanup

## Bucket Cleanup CLI Commands

### List Stack Buckets Contents

In [ ]:
# print(envResourceMap['Object Storage Buckets'])
for buckets in envResourceMap['Object Storage Buckets']:
    for bucket in buckets.keys():
#         print('oci os object list -bn ' +   bucket + ' -ns ' +envResourceMap['namespace']  +   config)
        command = 'oci  os object list -bn ' +   bucket + ' -ns ' +envResourceMap['namespace']  +  config
        print('Command: \n', command)
        try:
            stdout, stderr = runCLI(command)
        except:
            print('Errorx', stderr)
        print('Error', stdout)
        contents = json.loads(stdout)
        print('\nContents in Bucket: ' +  bucket  + '\n' )
        totalSize = 0
        for file in contents['data']:
            fileSize = file['size']/1000 
            totalSize += fileSize
#             format_float = "{:.2f}".format(float)
            print(file['name'] + "  size: {:.2f} KB".format(fileSize)    )
        print('---\n')
        print('Total Size of Bucket is: '  + "{:.0f} KB".format(totalSize)   )
        print('\n-------------\n')
            
        

### Delete bulk
<div class="alert alert-block alert-danger">
<b>Danger:</b> Running Bulk Version Delete will permanently remove All Files.  Use caution when using.
</div>

In [ ]:
for buckets in envResourceMap['Object Storage Buckets']:
    for bucket in buckets.keys():
        command = 'oci  os object  bulk-delete-versions --force -bn ' +   bucket + \
                ' -ns ' +envResourceMap['namespace']  +  config
        print('\nAre You Sure You Want to Delete Bucket - ', bucket + ' (Y/n) ??')
        a = input()
        if a != 'Y':
            print('Aborting Delete !! of Bucket - ', bucket)
            continue
        print('Command: \n', command)
        try:
            stdout, stderr = runCLI(command)
            print('\nCompleted Bulk Version Delete of Bucket: ', bucket)
            contents = json.loads(stdout)
        except:
            print('\nError', stderr)      
print('Done !!')